In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product
from fredapi import Fred
import warnings
from dotenv import load_dotenv
import os

warnings.filterwarnings('ignore')

def arima_grid_search(data, p_range, d_range, q_range, P_range, D_range, Q_range, s_range):
    best_aic = np.inf
    best_params = None
    
    for p, d, q, P, D, Q, s in product(p_range, d_range, q_range, P_range, D_range, Q_range, s_range):
        try:
            model = SARIMAX(data, order=(p, d, q), seasonal_order=(P, D, Q, s))
            results = model.fit(disp=False)
            aic = results.aic
            if aic < best_aic:
                best_aic = aic
                best_params = (p, d, q, P, D, Q, s)
            print(f'ARIMA{(p,d,q)}x{(P,D,Q,s)} - AIC:{aic}')
        except:
            continue
    
    print(f'Best ARIMA{best_params} - AIC:{best_aic}')
    return best_params

if __name__ == "__main__":
    # Fetch data
    # Load API_key from .env
    load_dotenv()
    API_KEY = os.getenv('API_KEY')

    # Initialize Fred API with your API key
    fred = Fred(api_key=API_KEY)

    data = fred.get_series('DGS10')
    data = pd.DataFrame(data, columns=['Interest Rate'])
    data.index.name = 'Date'
    data = data[(data.index >= '2010-01-01') & (data.index < '2020-01-01')]
    data.dropna(inplace=True)

    # Set up the parameter grid
    p_range = range(0, 1)
    d_range = range(0, 2)
    q_range = range(0, 1)
    P_range = range(0, 1)
    D_range = range(0, 1)
    Q_range = range(0, 1)
    s_range = [0, 12]  # 0 for non-seasonal, 12 for yearly seasonality

    # Perform grid search
    best_params = arima_grid_search(data['Interest Rate'], 
                                    p_range, d_range, q_range, 
                                    P_range, D_range, Q_range, s_range)

    print("Best parameters (p, d, q, P, D, Q, s):", best_params)

ARIMA(0, 0, 0)x(0, 0, 0, 0) - AIC:11618.142134137295
ARIMA(0, 0, 0)x(0, 0, 0, 12) - AIC:11618.142134137295
ARIMA(0, 1, 0)x(0, 0, 0, 0) - AIC:-8006.8357387368305
ARIMA(0, 1, 0)x(0, 0, 0, 12) - AIC:-8006.8357387368305
Best ARIMA(0, 1, 0, 0, 0, 0, 0) - AIC:-8006.8357387368305
Best parameters (p, d, q, P, D, Q, s): (0, 1, 0, 0, 0, 0, 0)
